## Loading functions and libraries

In [1]:
import numpy as np
from sklearn.cluster import KMeans
# from functions import load_data_CMU, load_data, connect_neighbors, search_and_evaluate, analysis, generate_graph_html, mean_squared_error, mean_pixelwise_joint_squared_error, percentage_correct_keypoints, sliding_window
import time
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error
import time
import numpy as np
from sklearn.neighbors import KDTree
from sklearn.metrics import mean_squared_error
import time
import numpy as np
import os
import sys
import numpy as np
import librosa
from numba import jit
from matplotlib import patches
import libfmp.b
import libfmp.c3
import libfmp.c7
import networkx as nx
from dtw import *
from concurrent.futures import ThreadPoolExecutor, as_completed
import matplotlib.pyplot as plt

import h5py

query = None
dataset = None
matrix = None
indices = None
motion = None
data_eval = None

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



## Loading data

In [2]:
def load_data(query_path,dataset_path):
    mat_file = h5py.File(query_path, 'r')
    mat_file2 = h5py.File(dataset_path, 'r')
    Q = mat_file['queryDataset']['pos']
    D = mat_file2['wholeDataset']['pos']
    Qnp = np.array(Q)
    Dnp = np.array(D)
    query = Qnp
    dataset = Dnp
    return query, dataset


def load_data_CMU(query_path,dataset_path):
    query = np.load(query_path)
    dataset = np.load(dataset_path)

    return query, dataset

In [3]:
# Step 1

s_load = time.time()
# query, dataset = load_data('queryDataset.mat', 'wholeDataset.mat')
query, dataset = load_data_CMU('query.npy','Dataset.npy')
e_load = time.time() - s_load

query_shape = str(query.shape) if query is not None else None
dataset_shape = str(dataset.shape) if dataset is not None else None
print("Query shape: ",query_shape)
print("Dataset shape: ",dataset_shape)

print("Time taken to load data: ",e_load,' seconds')

Query shape:  (2756, 93)
Dataset shape:  (1132475, 93)
Time taken to load data:  5.2776031494140625  seconds


## Nearest Neighbor Search

In [4]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import time

def mean_squared_error(vec1, vec2):
    """Compute the Mean Squared Error (MSE) between two vector arrays."""
    return np.mean((vec1 - vec2) ** 2)

def search_and_evaluate(query, dataset, K, algorithm='auto', metric='minkowski', p=2, radius=1.0):
    """
    Perform K-nearest neighbor search and evaluate using given parameters.

    Parameters:
    - query: np.ndarray, the query matrix.
    - dataset: np.ndarray, the dataset matrix.
    - K: int, the number of nearest neighbors to retrieve.
    - algorithm: str, algorithm to compute the nearest neighbors ('auto', 'ball_tree', 'kd_tree', 'brute').
    - metric: str, the distance metric to use for the tree.
    - p: int, power parameter for the Minkowski metric.
    - radius: float, radius for radius neighbors search.

    Returns:
    - indices: np.ndarray, the indices of the nearest neighbors.
    - data_eval: dict, dictionary containing evaluation metrics.
    - mean_mse: float, mean of the Mean Squared Errors.
    - mean_mpjse: float, mean of the Mean Per Joint Squared Errors.
    - mean_pck: float, mean of the Percentages of Correct Keypoints.
    - retrieval_time: float, the time taken for the retrieval.
    """
    # Initialize the NearestNeighbors model
    knn = NearestNeighbors(n_neighbors=K, algorithm=algorithm, leaf_size=30, metric=metric, p=p, radius=radius)
    
    # Fit the model on the dataset
    knn.fit(dataset)
    
    # Perform K-nearest neighbor search for the query matrix
    start_time = time.time()
    distances, indices = knn.kneighbors(query, n_neighbors=K)
    retrieval_time = time.time() - start_time
    
    # Initialize variables to store evaluation metrics
    mse_list = []
    mpjse_list = []
    pck_list = []
    
    # Calculate evaluation metrics for each query pose
    for i in range(len(query)):
        # Get the K-nearest neighbors from the dataset
        nearest_neighbors = dataset[indices[i]]
        
        # Calculate Mean Squared Error (MSE)
        mse = mean_squared_error(query[i], nearest_neighbors)
        mse_list.append(mse)
        
        # Calculate Mean Per Joint Squared Error (MPJSE)
        mpjse = np.mean(np.linalg.norm(query[i] - nearest_neighbors, axis=1))
        mpjse_list.append(mpjse)
        
        # Compute PCK (Percentage of Correct Keypoints)
        threshold = 30  # Set a threshold for correctness
        correct_keypoints = np.linalg.norm(query[i] - nearest_neighbors, axis=1) < threshold
        pck = np.sum(correct_keypoints) / len(correct_keypoints)
        pck_list.append(pck)
        
    # Calculate the mean of each evaluation metric
    mean_mse = np.mean(mse_list)
    mean_mpjse = np.mean(mpjse_list)
    mean_pck = np.mean(pck_list)
    
    # Prepare evaluation data for output
    data_eval = {
        "Mean Squared Error (MSE)": mean_mse,
        "Mean Per Joint Squared Error (MPJSE)": mean_mpjse,
        "Percentage of Correct Keypoints (PCK)": mean_pck,
        "Retrieval Time": retrieval_time
    }
    
    # Print and return results
    print("Evaluation Results:")
    for metric_name, value in data_eval.items():
        print(f"{metric_name}: {value}")
    
    return indices, data_eval, mean_mse, mean_mpjse, mean_pck, retrieval_time



In [5]:
# Example usage
K = 128  # Number of nearest neighbors to retrieve

# Perform search and evaluation with different algorithms and metrics
algorithms = ['auto', 'ball_tree', 'kd_tree', 'brute']
metrics = ['minkowski', 'euclidean', 'manhattan','l1','l2','cosine']
radius_values = [1.0, 2.0, 5.0]

algorithm = algorithms[2]
metric = metrics[0]
radius = radius_values[1]

print(f"\nUsing algorithm={algorithm}, metric={metric}, radius={radius}")
indices, data_eval, mean_mse, mean_mpjse, mean_pck, retrieval_time = search_and_evaluate(query, dataset, K, algorithm=algorithm, metric=metric, p=2, radius=radius)

# Retrieve the top 128 nearest neighbors' data from the dataset
top_neighbors_data = dataset[indices[:, :K]]

print(top_neighbors_data.shape)



Using algorithm=kd_tree, metric=minkowski, radius=2.0
Evaluation Results:
Mean Squared Error (MSE): 4.0240654673736875
Mean Per Joint Squared Error (MPJSE): 18.48859606773928
Percentage of Correct Keypoints (PCK): 0.9729907474600871
Retrieval Time: 49.08016061782837
(2756, 128, 93)


## K-Means Clustering to Reduce data

In [7]:
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial import distance
import os

# Set the environment variable to avoid memory leak on Windows with MKL
os.environ["OMP_NUM_THREADS"] = "1"

# Parameters
kmeans_nn = 64  # Desired number of nearest neighbors after clustering

# Function to calculate reconstruction error
def calculate_error(original, clustered):
    return np.linalg.norm(original - clustered)

# Step 1: Perform K-means clustering on each row's nearest neighbors and select nearest points
new_data = []

K_M_C_time = time.time()  # noting time for clustering process

for i in range(top_neighbors_data.shape[0]):  # Iterate over each row
    # Extract the nearest neighbors for the current row
    nearest_neighbors_data = top_neighbors_data[i]

    # Ensure kmeans_nn is less than or equal to the number of nearest neighbors
    if kmeans_nn > nearest_neighbors_data.shape[0]:
        kmeans_nn = nearest_neighbors_data.shape[0]

    # Perform K-means clustering on the nearest neighbors
    kmeans_model_nn = KMeans(n_clusters=kmeans_nn, random_state=0, n_init=10)
    kmeans_labels_nn = kmeans_model_nn.fit_predict(nearest_neighbors_data)

    # Calculate the cluster centers (new nearest neighbors)
    centroids = kmeans_model_nn.cluster_centers_

    # Find the nearest points in the original data to the centroids
    nearest_points = []
    for centroid in centroids:
        distances = distance.cdist([centroid], nearest_neighbors_data, 'euclidean')
        nearest_point_index = np.argmin(distances)
        nearest_points.append(nearest_neighbors_data[nearest_point_index])

    new_data.append(nearest_points)

# Convert list to numpy array
new_data = np.array(new_data)

K_M_C_time_e = time.time() - K_M_C_time  # noting time for clustering process

print("New data shape:", new_data.shape)
print("Total clustering time: ",K_M_C_time_e)


## Graph Construction to extract global alignments

In [ ]:
def find_all_shortestP(matrix):
    # Create graph and connect neighbors
    graph = connect_neighbors(matrix)
    
    total_costs = []
    S_neighbors = []
    ASP = []
    
    start_t = time.time()
    print("Shortest path search started now..")
    
    for successor in graph.successors('S'):
        S_neighbors.append(successor)
    
        # Find all shortest paths from 'S' to 'E'
        all_shortest_paths = list(nx.all_shortest_paths(graph, source=successor, target='E', weight='weight'))
        ASP.extend(all_shortest_paths)  # Extend instead of append to flatten the list
    
        # Compute the total cost of each path
        for path in all_shortest_paths:
            total_cost = sum(graph[u][v]['weight'] for u, v in zip(path[:-1], path[1:]))
            total_costs.append(total_cost)
    
    
    # Ensure the lengths of total_costs and ASP are the same
    assert len(total_costs) == len(ASP), "Mismatch between total costs and paths"
    
    # Find the index of the shortest path with the lowest cost
    min_cost_index = total_costs.index(min(total_costs))
    
    return ASP, total_costs, min_cost_index, graph



def connect_neighbors(matrix):
    """Connect neighboring nodes in a grid-like manner."""
    rows, cols, h = matrix.shape
    
    # Create a directed graph
    G = nx.DiGraph()
    
    # Dictionary for node indexing
    node_indexes = {}
    
    # Add nodes and populate node indexing dictionary
    node_count = 0
    for i in range(rows):
        for j in range(cols):
            node = (i, j)
            G.add_node(node)
            node_indexes[node] = node_count
            node_count += 1
    
    # Connect nodes according to grid-like logic
    for i in range(rows):
        for j in range(cols):
            current_node = (i, j)
            current_node_index = node_indexes[current_node]
            
            # Connect to the right neighbor
            if j < cols - 1:
                neighbor_node = (i, j + 1)
                neighbor_node_index = node_indexes[neighbor_node]
                edge_weight = np.mean(np.square(matrix[i, j] - matrix[i, j + 1]))  # Example of edge weight calculation
                G.add_edge(current_node, neighbor_node, weight=edge_weight)
            
            # Connect to the bottom neighbor
            if i < rows - 1:
                neighbor_node = (i + 1, j)
                neighbor_node_index = node_indexes[neighbor_node]
                edge_weight = np.mean(np.square(matrix[i, j] - matrix[i + 1, j]))  # Example of edge weight calculation
                G.add_edge(current_node, neighbor_node, weight=edge_weight)
            
            # Connect diagonally to the bottom-right neighbor
            if i < rows - 1 and j < cols - 1:
                neighbor_node = (i + 1, j + 1)
                neighbor_node_index = node_indexes[neighbor_node]
                edge_weight = np.mean(np.square(matrix[i, j] - matrix[i + 1, j + 1]))  # Example of edge weight calculation
                G.add_edge(current_node, neighbor_node, weight=edge_weight)
            
            # Connect diagonally to the bottom-left neighbor
            if i < rows - 1 and j > 0:
                neighbor_node = (i + 1, j - 1)
                neighbor_node_index = node_indexes[neighbor_node]
                edge_weight = np.mean(np.square(matrix[i, j] - matrix[i + 1, j - 1]))  # Example of edge weight calculation
                G.add_edge(current_node, neighbor_node, weight=edge_weight)
    
    # Add starting node 'S' at positions (0, 1), (1, 0), (2, 0), ...
    for i in range(rows):
        start_node = (i, 0)
        G.add_edge('S', start_node, weight=1)
    
    # Add end node 'E' at the last element of each row
    for i in range(rows):
        end_node = (i, cols - 1)
        G.add_edge(end_node, 'E', weight=1)
    
    return G


In [ ]:
import matplotlib.pyplot as plt
# from functions import  find_all_shortest_paths,load_data_CMU,load_data, connect_neighbors, search_and_evaluate, analysis, generate_graph_html, mean_squared_error,mean_pixelwise_joint_squared_error, percentage_correct_keypoints, sliding_window
import networkx as nx
import time

def graph_construction(matrix):
    # Generate the graph based on the options
    # Return HTML code for the graph
    # part 3 Lazy neighborhood graph construction
    # Draw the graph
    matrix = np.array(matrix)
    plt.figure(figsize=(40, 40))

    # Positioning nodes
    pos = {'S': (-1, 0)}  # Set position for 'S' node
    for i, example_index in enumerate(range(matrix.shape[0])):
        for j, neighbor_index in enumerate(range(matrix.shape[1])):
            pos[(example_index, neighbor_index)] = (j, -i)  # Position each example and its neighbors in a square-like shape
    pos['E'] = (matrix.shape[1] + 1, 0)  # Set position for 'E' node

    # Create graph and connect neighbors
    graph = connect_neighbors(matrix)

    
    start_t = time.time()
    # part 4 Shortest path search
    # Find all shortest paths, display total cost of each, and highlight the one with the lowest cost
    all_shortest_paths, total_costs, min_cost_index, graph = find_all_shortestP(matrix)
    
    endtime = time.time()
    
    print("Time taken to find alignements using graph:", endtime - start_t)

#     # Highlight the shortest path
    shortest_path_nodes = all_shortest_paths[min_cost_index]

    # Drawing nodes and edges
    nx.draw_networkx_nodes(graph, pos, node_size=1000, node_color="skyblue")
    nx.draw_networkx_edges(graph, pos, edge_color="gray")
    nx.draw_networkx_labels(graph, pos, font_size=12, font_weight="bold")
    nx.draw_networkx_edge_labels(graph, pos, edge_labels={(u, v): round(d["weight"], 2) for u, v, d in graph.edges(data=True)})

#     Drawing the shortest path nodes with a different color
    nx.draw_networkx_nodes(graph, pos, nodelist=shortest_path_nodes, node_color="red", node_size=1000)

    plt.title("Interconnected Graph with Shortest Path Highlighted")
    plt.axis("off")  # Turn off axis
    
    plt.show()
    
#     Save the graph as an image
#     plt.savefig("./GGG.png")
#     plt.close()  # Close the plot to avoid displaying it

    # part 5 DTW analysis
    retireved_motion = shortest_path_nodes[:-1]

#     print(shortest_path_nodes)

    motion = []
    for x, y in retireved_motion:
        print(x, y)
        motion.append(matrix[x][y])

    motion = np.array(motion)

    return endtime - start_t,'<img src="./graph.png" alt="Graph">' , shortest_path_nodes, total_costs[min_cost_index], motion 


## Graph Search using djistra shortest path

In [ ]:

# Step 4
# GGH(mat2)
t2, graph_html, shortest_path_nodes, minimum_cost, motion2 = graph_construction(new_data[:10,:10]) #Pose Quest Method


In [ ]:
print("Time taken using PoseQuest Method: ", t2)

## Retrieved Motion Comparison Using Evaluation Metrics

In [ ]:
# pose quest method motion retrieval


def mean_squared_error(vec1, vec2):
    """Compute the Mean Squared Error (MSE) between two vector arrays."""
    return np.mean((vec1 - vec2) ** 2)

def mean_pixelwise_joint_squared_error(vec1, vec2):
    """Compute the Mean Pixel-wise Joint Squared Error (MPJSE) between two vector arrays."""
    return np.mean(np.mean((vec1 - vec2) ** 2, axis=1))

def percentage_correct_keypoints(vec1, vec2, threshold):
    """Compute the Percentage of Correct Keypoints (PCK) between two vector arrays."""
    distances = np.linalg.norm(vec1 - vec2, axis=1)
    return np.mean(distances < threshold) * 100

vec_array1 = motion2
vec_array2 = query[:len(motion2)]
mse = mean_squared_error(vec_array1, vec_array2)
mpjse = mean_pixelwise_joint_squared_error(vec_array1, vec_array2)
threshold = 10  # Set threshold for PCK
pck = percentage_correct_keypoints(vec_array1, vec_array2, threshold)


print("Mean Squared Error (MSE):", mse)
print("Mean Pixel-wise Joint Squared Error (MPJSE):", mpjse)
print("Percentage of Correct Keypoints (PCK):", pck)


In [ ]:
np.save("extracted_motion_own.npy",motion2)